In [11]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
from dataclasses import asdict

import pandas as pd

from juddges.settings import NSA_DATA_PATH
import polars as pl
import pandas as pd
from mpire import WorkerPool

lf = pl.scan_parquet(NSA_DATA_PATH / "pages" / "pages_chunk_0.parquet")

In [13]:
df = lf.collect()

In [14]:
# df[2]["page"]

In [15]:
from juddges.data.nsa.extractor import NSADataExtractor, ORDER

extractor = NSADataExtractor()
df = extractor.extract_data_from_pages_to_df(df["page"], df["doc_id"])

/home/albert/miniconda3/envs/JuDDGES/lib/python3.11/site-packages/mpire/pool.py:692: RuntimeWarning: Failed to obtain length of iterable when chunk size or number of splits is None. Chunk size is set to 4. Remedy: either provide an iterable with a len() function or specify iterable_len in the function call
  n_tasks, max_tasks_active, chunk_size, progress_bar, progress_bar_options = check_map_parameters(
100%|██████████| 50000/50000 [00:33<00:00, 1429.11it/s]


In [16]:
assert len(df.columns) == len(ORDER)

In [20]:
len(ORDER)

22

In [18]:
df.columns

Index(['id', 'Docket number', 'Type of decision', 'Finality',
       'The day of the judgment', 'Date of submission', 'Court', 'Judges',
       'Presiding judge', 'Judge rapporteur',
       'Type of case with the detailed description', 'Keywords',
       'Related docket numbers', 'Challenged authority',
       'Nature of the verdict', 'The cited provisions',
       'Published in official collection of judgments Jurisprudence of the Voivodeship Administrative Courts and the Supreme Administrative Court',
       'Information on glosa(s)', 'Theses', 'Sentence of the judgment',
       'Reasons for judgment', 'Dissenting opinion'],
      dtype='object')

In [21]:
df[:1000]

,id,Docket number,Type of decision,Finality,The day of the judgment,Date of submission,Court,Judges,Presiding judge,Judge rapporteur,...,Related docket numbers,Challenged authority,Nature of the verdict,The cited provisions,Published in official collection of judgments Jurisprudence of the Voivodeship Administrative Courts and the Supreme Administrative Court,Information on glosa(s),Theses,Sentence of the judgment,Reasons for judgment,Dissenting opinion
0,/doc/BC8847077B,II SA/Ol 1164/16,Postanowienie WSA w Olsztynie,orzeczenie prawomocne,2016-10-26,2016-09-21,Wojewódzki Sąd Administracyjny w Olsztynie,[S. Beata Jezielska],S. Beata Jezielska,S. Beata Jezielska,...,NaN,Dyrektor Izby Celnej,[Odrzucono skargę],"[{'Journal of laws': 'Dz.U. 2016 poz 718', 'Ar...",NaN,NaN,NaN,Wojewódzki Sąd Administracyjny w Olsztynie w s...,Spółka A z siedzibą w W - reprezentowana przez...,NaN
1,/doc/C8D885030F,III SA/Po 595/21,Wyrok WSA w Poznaniu,orzeczenie nieprawomocne,2021-10-14,2021-04-02,Wojewódzki Sąd Administracyjny w Poznaniu,"[Izabela Paluszyńska, Piotr Ławrynowicz, Walen...",Izabela Paluszyńska,Izabela Paluszyńska,...,NaN,Dyrektor Oddziału Regionalnego Agencji Restruk...,[Oddalono skargę],"[{'Journal of laws': 'Dz.U. 2019 poz 2325', 'A...",NaN,NaN,NaN,Dnia 14 października 2021 roku Wojewódzki Sąd ...,Decyzją z [...] lutego 2020r. Nr [...] Kierown...,NaN
2,/doc/6447E5BA57,II SA/Op 8/05,Wyrok WSA w Opolu,orzeczenie prawomocne,2005-10-13,2005-01-12,Wojewódzki Sąd Administracyjny w Opolu,"[Grażyna Jeżewska, Jerzy Krupiński, Teresa Cisyk]",Jerzy Krupiński,Teresa Cisyk,...,NaN,Komendant Policji,[Uchylono decyzję I i II instancji],[{'Journal of laws': 'Dz.U. 2002 nr 7 poz. 58'...,ONSAiWSA 2006 4 poz. 116,NaN,Przepisy art. 32 ust. 1 i 2 ustawy z dnia 23 m...,Wojewódzki Sąd Administracyjny w Opolu w skład...,"Komendant Miejski Policji w O., rozkazem perso...",NaN
3,/doc/B4984CED3A,II FSK 1938/08,Wyrok NSA,orzeczenie prawomocne,2010-04-02,2008-11-12,Naczelny Sąd Administracyjny,"[Bogusław Woźniak, Grzegorz Borkowski, Stefan ...",Grzegorz Borkowski,Bogusław Woźniak,...,"[{'Docket number': 'I SA/Wr 183/08', 'Type of ...",Dyrektor Izby Skarbowej,[Oddalono skargę kasacyjną],[{'Journal of laws': 'Dz.U. 1997 nr 123 poz 77...,NaN,NaN,NaN,Naczelny Sąd Administracyjny w składzie: Przew...,Wojewódzki Sąd Administracyjny we Wrocławiu wy...,NaN
4,/doc/21CBD4642A,III SA/Wa 469/14,Postanowienie WSA w Warszawie,None,2015-05-19,2014-02-10,Wojewódzki Sąd Administracyjny w Warszawie,[Maciej Kurasz],Maciej Kurasz,Maciej Kurasz,...,NaN,Dyrektor Izby Skarbowej,[Odrzucono skargę kasacyjną],"[{'Journal of laws': 'Dz.U. 2012 poz 270', 'Ar...",NaN,NaN,NaN,Dnia 19 maja 2015 r. Wojewódzki Sąd Administra...,Wojewódzki Sąd Administracyjny w Warszawie wyr...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,/doc/F716824513,I SA/Wa 640/17,Postanowienie WSA w Warszawie,orzeczenie prawomocne,2017-06-22,2017-04-26,Wojewódzki Sąd Administracyjny w Warszawie,[Elżbieta Sobielarska],Elżbieta Sobielarska,Elżbieta Sobielarska,...,NaN,Samorządowe Kolegium Odwoławcze,[Odrzucono skargę],NaN,NaN,NaN,NaN,Wojewódzki Sąd Administracyjny w Warszawie w s...,Pismem z dnia [...] marca 2017 r. M. O. i K. O...,NaN
996,/doc/DE05AA888C,I SA/Wa 660/18,Postanowienie WSA w Warszawie,None,2018-10-30,2018-04-13,Wojewódzki Sąd Administracyjny w Warszawie,[Iwona Kosińska],Iwona Kosińska,Iwona Kosińska,...,"[{'Docket number': 'I OSK 945/22', 'Type of de...",Inne,[Wstrzymano wykonanie zaskarżonej decyzji],"[{'Journal of laws': 'Dz.U. 2018 poz 1302', 'A...",NaN,NaN,NaN,"Wojewódzki Sąd Administracyjny w Warszawie, w ...",R. K. wniósł do Wojewódzkiego Sądu Administrac...,NaN
997,/doc/FC756C2C43,I GZ 533/13,Postanowienie NSA,orzeczenie prawomocne,2013-12-10,2013-11-28,Naczelny Sąd Administracyjny,[Wojciech Kręcisz],Wojciech Kręcisz,Wojciech Kręcisz,...,"[{'Docket number': 'III SA/Gl 1494/12', 'Type ...",Dyrektor Izby Celnej,[Uchylono zaskarżone postanowienie i przekaz